In [3]:
import os
import glob
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
dataset_dir = "/content/vision_transformer"

class_names = ["Pothole", "speedbreaker"]


images = []
labels = []


for split in ["train", "test", "valid"]:
    split_dir = os.path.join(dataset_dir, split)


    for class_name in class_names:
        class_dir = os.path.join(split_dir, class_name)


        image_files = glob.glob(os.path.join(class_dir, "*.jpg"))


        for image_file in image_files:
            img = load_img(image_file, target_size=(224, 224))
            img = img_to_array(img)
            img = img / 255.0
            images.append(img)

            label = class_names.index(class_name)
            labels.append(label)

images = np.array(images)
labels = np.array(labels)


num_classes = len(class_names)
one_hot_labels = to_categorical(labels, num_classes=num_classes)

X_train, X_test, y_train, y_test = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten,Dense
from tensorflow.keras.models import Model

input_shape=(224,224,3)
def create_vgg16():
  inputs=Input(input_shape)

  #Block 1
  x= Conv2D(64,(3,3),activation='relu', padding='same')(inputs)
  x= Conv2D(64,(3,3), activation='relu',padding='same')(x)
  x= MaxPooling2D((2,2),strides=(2,2))(x)

  #Block 2
  x= Conv2D(128, (3,3), activation='relu', padding='same')(x)
  x= Conv2D(128, (3,3), activation='relu', padding='same')(x)
  x= MaxPooling2D((2,2),strides=(2,2))(x)

  #Block3
  x= Conv2D(256, (3,3), activation='relu', padding='same')(x)
  x= Conv2D(256, (3,3), activation='relu', padding='same')(x)
  x= Conv2D(256, (3,3), activation='relu', padding='same')(x)
  x= MaxPooling2D((2,2),strides=(2,2))(x)

  #Block 4
  x= Conv2D(512, (3,3),  activation='relu', padding='same')(x)
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)
  x= MaxPooling2D((2,2), strides=(2,2))(x)

  # Block 5
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  #Fully Connected Layers
  x= Flatten()(x)
  x= Dense(4096, activation='relu')(x)
  x= Dense(4096, activation='relu')(x)

  # Output layer for our classes (2 classes: Pothole and speedbreaker)
  outputs= Dense(2, activation='softmax')(x)

  #Creating the model
  model= Model(inputs, outputs)

  return model


In [ ]:
vgg16_model = create_vgg16()

vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

vgg16_model.fit(X_train, y_train, epochs=50, batch_size=32)

vgg16_model.save("vgg16.h5")

In [ ]:
def create_vgg19():
  inputs=Input(input_shape)

  #Block 1
  x= Conv2D(64,(3,3), activation='relu',padding='same')(inputs)#1
  x= Conv2D(64,(3,3), activation='relu', padding='same')(x)#2
  x= MaxPooling2D((2,2), strides=(2,2))(x)

  #Block 2
  x= Conv2D(128, (3,3), activation='relu', padding='same')(x)#3
  x= BatchNormalization()(x)
  x= Conv2D(128, (3,3), activation='relu', padding='same')(x)#4
  x= MaxPooling2D((2,2), strides=(2,2))(x)

  #Block 3
  x= Conv2D(256, (3,3), activation='relu', padding='same')(x)#5
  x= Conv2D(256,(3,3), activation='relu', padding='same')(x)#6
  x= Conv2D(256, (3,3), activation='relu', padding='same')(x)#7
  x= Dropout(0.3)(x)
  x= Conv2D(256, (3,3), activation='relu', padding='same')(x)#8
  x= MaxPooling2D((2,2), strides=(2,2))(x)

  #Block 4
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#9
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#10
  x= BatchNormalization()(x)
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#11
  x= Dropout(0.2)(x)
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#12
  x= MaxPooling2D((2,2), strides=(2,2))(x)

  #Block 5
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#13
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#14
  x= Dropout(0.1)(x)
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#15
  x= BatchNormalization()(x)
  x= Conv2D(512, (3,3), activation='relu', padding='same')(x)#16
  x= MaxPooling2D((2,2), strides=(2,2))(x)

  #Block 6
  x= Flatten()(x)#17
  x= Dense(4096, activation='relu')(x)#18
  x= Dense(4096, activation='relu')(x)#19

  outputs= Dense(2, activation='softmax')(x)

  model= Model(inputs, outputs)

  return model

In [ ]:
vgg19= create_vgg19()

vgg19.summary()

vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg19.save('VGG19.h5')

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
model_1 = load_model('/content/vgg/VGG19.h5')
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='name_of_model_1')
model_2 = load_model('/content/vgg/vgg16.h5')
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='name_of_model_2')
models = [model_1, model_2]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 name_of_model_1 (Functiona  (None, 2)                    1395830   ['input_1[0][0]']             
 l)                                                       42                                      
                                                                                                  
 name_of_model_2 (Functiona  (None, 2)                    1342687   ['input_1[0][0]']             
 l)                                                       38                                      
                                                                                           

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(ensemble_model, to_file='ensemble_model_structure.png', show_shapes=True, show_layer_names=True)

In [ ]:
ensemble_model.fit(X_train, y_train, epochs=10, batch_size=32)

In [8]:
from sklearn.metrics import confusion_matrix, auc, classification_report, accuracy_score, roc_curve, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
y_pred = ensemble_model.predict(X_test)
fpr, tpr, _ = roc_curve(y_test[:, 0], y_pred[:, 0])
roc_auc = auc(fpr, tpr)

8/8 [==============================] - 5s 700ms/step


In [ ]:
y_true = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
confusion_mtx = confusion_matrix(y_true, y_pred_class)
disp = ConfusionMatrixDisplay(confusion_mtx, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, values_format="d")
plt.tight_layout()
plt.show()

In [11]:
accuracy_score(y_true,y_pred_class)

0.5102880658436214

In [12]:
test_loss, test_acc = ensemble_model.evaluate(X_test, y_test)

8/8 [==============================] - 3s 261ms/step - loss: 0.7095 - accuracy: 0.5103


In [ ]:
test_loss, test_acc

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
y_pred = ensemble_model.predict(X_test)
fpr, tpr, _ = roc_curve(y_test[:, 0], y_pred[:, 0])
roc_auc = auc(fpr, tpr)

# Plotting ROC curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

image_path = "/content/vision_transformer/test/speedbreaker/speedbreaker401.jpg"
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)


predictions = ensemble_model.predict(img_array)


class_names = ["Pothole", "Speedbreaker"]

predicted_class_index = np.argmax(predictions)
predicted_class = class_names[predicted_class_index]
confidence_score = predictions[0][predicted_class_index]
plt.figure(figsize=(3, 5))
plt.subplot(2, 1, 1)
plt.imshow(img)
plt.axis('off')
plt.subplot(2, 1, 2)
text = f"Predicted class: {predicted_class}\nConfidence: {confidence_score:.2f}"
plt.text(0.5, 0.5, text, fontsize=12, ha='center', va='center')
plt.axis('off')

plt.tight_layout()
plt.show()